<a href="https://colab.research.google.com/github/rohmadyudiutomo/p300/blob/main/p300_model_by_Deepchem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DEEPCHEM for preparing p300 activity prediction
must run in GPU

In [1]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 320.4MB 52kB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [2]:
# Installing conda
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  16859      0 --:--:-- --:--:-- --:--:-- 16859


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [3]:
# Installing Deepchem
!pip install --pre deepchem
import deepchem
deepchem.__version__

     |████████████████████████████████| 409kB 7.8MB/s 
  Created wheel for deepchem: filename=deepchem-2.4.0rc1.dev20201118013711-cp36-none-any.whl size=516159 sha256=1c5995bb1c8045f89688d5aa0d98b163a483b8413bb4b32ba6d65a7b8ddd4c50
  Stored in directory: /root/.cache/pip/wheels/d0/62/46/62d0ff22abaf0778aa1ef94cde4282cb45fcb190f5cdd4ec75
Successfully built deepchem


'2.4.0-rc1.dev'

In [86]:
# Getting the dataset
!wget https://raw.githubusercontent.com/rohmadyudiutomo/p300/main/Cur%20der%20p300%20from%20Yang%20et%20al%20mix%20cur%20filter.csv
from deepchem.utils.save import load_from_disk
dataset_file= "Cur der p300 from Yang et al mix cur filter.csv"

# Loading the data from the CSV file
loader = deepchem.data.CSVLoader(tasks=["pIC50"], 
                                 smiles_field="SMILES", 
                                 featurizer=deepchem.feat.ConvMolFeaturizer())
# Featurizing the dataset with ConvMolFeaturizer
dataset = loader.featurize(dataset_file)

--2020-11-18 03:18:09--  https://raw.githubusercontent.com/rohmadyudiutomo/p300/main/Cur%20der%20p300%20from%20Yang%20et%20al%20mix%20cur%20filter.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13978 (14K) [text/plain]
Saving to: ‘Cur der p300 from Yang et al mix cur filter.csv.1’

Cur der p300 from Y 100%[===================>]  13.65K  --.-KB/s    in 0s      

2020-11-18 03:18:10 (105 MB/s) - ‘Cur der p300 from Yang et al mix cur filter.csv.1’ saved [13978/13978]



smiles_field is deprecated and will be removed in a future version of DeepChem.Use feature_field instead.
/usr/local/lib/python3.6/dist-packages/deepchem/data/data_loader.py:162: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  "featurize() will be removed in DeepChem 3.0", FutureWarning)


In [87]:
# Splitter splits the dataset 
# In this case it's is an equivalent of train_test_split from sklearn
splitter = deepchem.splits.RandomSplitter()
# frac_test is 0.01 because we only use a train and valid as an example
train, valid, _ = splitter.train_valid_test_split(dataset,
                                                  frac_train=0.7,
                                                  frac_valid=0.29,
                                                  frac_test=0.01)
# Normalizer will normalize y values in the dataset
normalizer = deepchem.trans.NormalizationTransformer(transform_y=True, 
                                                     dataset=train, 
                                                     move_mean=True)
train = normalizer.transform(train)
test = normalizer.transform(valid)

In [88]:
print(f"Size of the training data: {len(train.ids)}")
print(f"Size of the validation data: {len(valid.ids)}")
print(test)

Size of the training data: 102
Size of the validation data: 43
<DiskDataset X.shape: (43,), y.shape: (43, 1), w.shape: (43, 1), ids: ['CC1=NN(c2ccc(C(=O)O)cc2)C(=O)/C1=C\\c1ccc(-c2cc(C)c(C)cc2[N+](=O)[O-])o1'
 'O=C1/C(=C/c2ccc(O)c(Br)c2)CCC/C1=C\\c1ccc(O)c(Br)c1'
 'CC(C)N(Cc1ccc2c(c1)C=CC(C)(C)O2)S(=O)(=O)c1ccccn1' ...
 'O=C(c1ccc(O)c(NC(=O)c2cccc3ccccc23)c1)c1ccc(O)c(NC(=O)c2cccc3ccccc23)c1'
 'CNC(=O)Nc1ccc2c(c1)CC[C@@]21OC(=O)N(CC(=O)N(Cc2ccccc2)[C@H](C)C2CC2)C1=O'
 'C[C@H](C1CC1)N(Cc1ccccc1)C(=O)CN1C(=O)N[C@@]2(CCc3ccccc32)C1=O'], task_names: ['pIC50']>


In [89]:
# GraphConvModel is a GNN model based on 
# Duvenaud, David K., et al. "Convolutional networks on graphs for
# learning molecular fingerprints."
from deepchem.models import GraphConvModel
graph_conv = GraphConvModel(1,
                            batch_size=30,
                            mode="regression")
# Defining metric. Closer to 1 is better
metric = deepchem.metrics.Metric(deepchem.metrics.pearson_r2_score)

In [98]:
# Fitting the model
graph_conv.fit(train, nb_epoch=100)

0.03761035919189453

In [99]:
# Reversing the transformation and getting the metric scores on 2 datasets
train_scores = graph_conv.evaluate(train, [metric], [normalizer])
valid_scores = graph_conv.evaluate(valid, [metric], [normalizer])
test_scores = graph_conv.evaluate(test, [metric], [normalizer])
print(f"Train Scores: {train_scores}")
print(f"Validation Scores: {valid_scores}")
print(f"test Scores: {test_scores}")

Train Scores: {'pearson_r2_score': 0.9602700641681027}
Validation Scores: {'pearson_r2_score': 0.4485035953563968}
test Scores: {'pearson_r2_score': 0.4485036017635912}


In [100]:
smiles = ['O=C(CC(/C=C/C1=CC=C(O)C(O)=C1)=O)/C=C/C2=CC(OC)=C(O)C=C2','O=C(CC(/C=C/C1=CC=C(O)C(O)=C1)=O)/C=C/C2=CC(OC(O)=O)=C(O)C=C2','O=C(CC(/C=C/C1=CC=C(O)C(O)=C1)=O)/C=C/C2=CC(Br)=C(O)C=C2','O=C(CC(/C=C/C1=CC=C(O)C(OC)=C1)=O)/C=C/C2=CC(OC(O)=O)=C(O)C=C2','O=C(CC(/C=C/C1=CC=C(O)C(OC)=C1)=O)/C=C/C2=CC(Br)=C(O)C=C2','O=C(CC(/C=C/C1=CC=C(O)C(Br)=C1)=O)/C=C/C2=CC(OC(O)=O)=C(O)C=C2','O=C(/C(CCC/1)=C/C2=CC(OC)=C(O)C=C2)C1=C\C3=CC=C(O)C(O)=C3','O=C(/C(CCC/1)=C/C2=CC(OC(O)=O)=C(O)C=C2)C1=C\C3=CC=C(O)C(O)=C3','O=C(/C(CCC/1)=C/C2=CC(Br)=C(O)C=C2)C1=C\C3=CC=C(O)C(O)=C3','O=C(/C(CCC/1)=C/C2=CC(OC(O)=O)=C(O)C=C2)C1=C\C3=CC=C(O)C(OC)=C3','O=C(/C(CCC/1)=C/C2=CC(Br)=C(O)C=C2)C1=C\C3=CC=C(O)C(OC)=C3','O=C(/C(CCC/1)=C/C2=CC(OC(O)=O)=C(O)C=C2)C1=C\C3=CC=C(O)C(Br)=C3','O=C(CC(/C=C/C1=CC=C(O)C(OC)=C1)=O)/C=C/C2=CC(OC)=C(O)C=C2','O=C1/C(CCC/C1=C\C2=CC=C(C(Br)=C2)O)=C/C3=CC(Br)=C(C=C3)O']

In [101]:
from rdkit import Chem
mols = [Chem.MolFromSmiles(s) for s in smiles]
featurizer = deepchem.feat.ConvMolFeaturizer()
x = featurizer.featurize(mols)

predicted_pIC50 = graph_conv.predict_on_batch(x)
predicted_pIC50

array([[-0.20332716],
       [-0.29523158],
       [ 0.27267447],
       [-0.17727722],
       [ 0.3948581 ],
       [ 0.5273888 ],
       [-1.5807843 ],
       [-1.1606088 ],
       [-0.6668961 ],
       [-1.1699895 ],
       [-0.47290358],
       [-0.3007403 ],
       [-0.28920236],
       [ 0.2197175 ]], dtype=float32)